### Monthly sales by Region & Category

In [0]:
%sql
CREATE OR REPLACE VIEW sales_db.gold.v_sales_monthly_region_category AS
SELECT
  d.month_start,
  g.region,
  p.category,
  p.sub_category,
  SUM(f.sales) AS total_sales,
  COUNT(*) AS order_lines,
  COUNT(DISTINCT f.customer_id) AS unique_customers,
  AVG(f.ship_delay_days) AS avg_ship_delay_days
FROM sales_db.gold.fact_sales f
JOIN sales_db.gold.dim_date d   ON f.date = d.date
JOIN sales_db.gold.dim_geo g    ON f.geo_id = g.geo_id
JOIN sales_db.gold.dim_product p ON f.product_id = p.product_id
GROUP BY 1,2,3,4;


In [0]:
%sql
select * from sales_db.gold.v_sales_monthly_region_category;

month_start,region,category,sub_category,total_sales,order_lines,unique_customers,avg_ship_delay_days
2017-06-01,East,Office Supplies,Binders,423.95,10,9,3.4
2017-02-01,East,Technology,Phones,130.46,2,2,5.5
2017-04-01,West,Furniture,Furnishings,191.71,4,4,5.25
2017-02-01,East,Office Supplies,Fasteners,45.20,2,2,6.5
2017-02-01,East,Office Supplies,Binders,398.35,1,1,7.0
2017-03-01,East,Office Supplies,Appliances,901.95,1,1,0.0
2017-12-01,West,Office Supplies,Envelopes,188.32,5,5,4.2
2016-08-01,West,Office Supplies,Appliances,239.43,2,2,5.5
2015-12-01,East,Furniture,Tables,713.61,3,3,5.0
2018-01-01,East,Office Supplies,Paper,166.68,5,4,4.4


### Top Products per Region


In [0]:
%sql
CREATE OR REPLACE VIEW sales_db.gold.v_top_products_by_region AS
SELECT *
FROM (
  SELECT
    g.region,
    p.product_id,
    p.product_name,
    p.category,
    SUM(f.sales) AS total_sales,
    DENSE_RANK() OVER (PARTITION BY g.region ORDER BY SUM(f.sales) DESC) AS sales_rank
  FROM sales_db.gold.fact_sales f
  JOIN sales_db.gold.dim_geo g     ON f.geo_id = g.geo_id
  JOIN sales_db.gold.dim_product p ON f.product_id = p.product_id
  GROUP BY 1,2,3,4
) t
WHERE sales_rank <= 10;


In [0]:
%sql
select * from sales_db.gold.v_top_products_by_region;

region,product_id,product_name,category,total_sales,sales_rank
Central,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,Technology,17499.95,1
Central,TEC-MA-10000822,Lexmark MX611dhe Monochrome Laser Printer,Technology,14279.91,2
Central,OFF-BI-10001120,Ibico EPK-21 Electric Binding System,Office Supplies,11339.94,3
Central,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,Office Supplies,10653.72,4
Central,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,Office Supplies,8710.33,5
Central,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,Furniture,6939.70,6
Central,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,Office Supplies,6100.75,7
Central,TEC-CO-10003763,Canon PC1060 Personal Laser Copier,Technology,4899.93,8
Central,TEC-PH-10004165,Mitel MiVoice 5330e IP Phone,Technology,4179.85,9
Central,OFF-SU-10002881,Martin Yale Chadless Opener Electric Letter Opener,Office Supplies,4164.05,10


### Category Contribution (share of total sales)

In [0]:
%sql
CREATE OR REPLACE VIEW sales_db.gold.v_sales_share_by_category AS
WITH cat AS (
  SELECT
    p.category,
    SUM(f.sales) AS total_sales
  FROM sales_db.gold.fact_sales f
  JOIN sales_db.gold.dim_product p ON f.product_id = p.product_id
  GROUP BY 1
),
tot AS (
  SELECT SUM(total_sales) AS grand_total FROM cat
)
SELECT
  c.category,
  c.total_sales,
  ROUND(100 * c.total_sales / t.grand_total, 2) AS pct_of_total_sales
FROM cat c CROSS JOIN tot t
ORDER BY c.total_sales DESC;


In [0]:
%sql
select * from sales_db.gold.v_sales_share_by_category;

category,total_sales,pct_of_total_sales
Technology,827061.64,36.60
Furniture,728091.00,32.22
Office Supplies,704693.04,31.18


### Best region overall

In [0]:
%sql

SELECT
  g.region,
  SUM(f.sales) AS total_sales
FROM sales_db.gold.fact_sales f
JOIN sales_db.gold.dim_geo g ON f.geo_id = g.geo_id
GROUP BY 1
ORDER BY total_sales DESC;


region,total_sales
West,710117.43
East,668643.98
Central,492646.90
South,388437.37


### Top 5 sub-categories

In [0]:
%sql

SELECT
  p.sub_category,
  SUM(f.sales) AS total_sales
FROM sales_db.gold.fact_sales f
JOIN sales_db.gold.dim_product p ON f.product_id = p.product_id
GROUP BY 1
ORDER BY total_sales DESC
LIMIT 5;


sub_category,total_sales
Phones,327782.49
Chairs,322541.38
Storage,218798.99
Tables,202810.77
Binders,199938.76


### Best month overall

In [0]:
%sql

SELECT
  d.month_start,
  SUM(f.sales) AS total_sales
FROM sales_db.gold.fact_sales f
JOIN sales_db.gold.dim_date d ON f.date = d.date
GROUP BY 1
ORDER BY total_sales DESC
LIMIT 1;


month_start,total_sales
2018-11-01,117393.76
